# "2019 September One Day Holding Long Short Portfolio BackTest"
> "2019 September Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [150]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [151]:
#hide_input
#df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
#df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')

In [152]:
#hide
#df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190901].loc[df_tes.Date<20191001].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [153]:
#hide
#df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_agg['cumreturn1']=0.0
#for l in range(len(df_agg)):
#    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
#df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
#df_ag['cumreturn1a']=0.0
#for l in range(len(df_ag)):
#    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [154]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [155]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-09-03,-1.0,-0.723623,1.007236,1.010842,1.009091
1,2019-09-04,-1.0,10.982052,0.896621,1.023993,1.023361
2,2019-09-05,-1.0,13.833619,0.772586,1.024925,1.026014
3,2019-09-05,1.0,-1.719902,0.759298,1.024925,1.026014
4,2019-09-06,-1.0,13.597485,0.656053,1.024829,1.027471
5,2019-09-09,-1.0,32.908813,0.440154,1.025159,1.030301
6,2019-09-09,1.0,2.904010,0.452936,1.025159,1.030301
7,2019-09-10,-1.0,61.744503,0.173273,1.032571,1.039016
8,2019-09-10,1.0,4.234528,0.180610,1.032571,1.039016
9,2019-09-11,-1.0,-3.842127,0.187549,1.035544,1.040755


In [156]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [157]:
#hide
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [158]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [159]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-09-05,1.0,-1.719902,0.982801,1.000911,1.002593
1,2019-09-09,1.0,2.904010,1.011342,1.001139,1.006782
2,2019-09-10,1.0,4.234528,1.054167,1.008377,1.015298
3,2019-09-11,1.0,-2.201336,1.030961,1.008377,1.015298


In [160]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [161]:
#hide
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [162]:
#hide
if len(df_a)>0:
    import altair as alt
    import datetime
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_a.Date.min()].loc[df_dow.Date<=df_a.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_a=pd.merge(df_a,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_a.Date.min()].loc[df_sp500.Date<=df_a.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_a=pd.merge(df_a,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    #df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    #df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    #df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    #df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    #df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [163]:
#hide_input
df_a

,Date,pred,labelr1,cumreturn1b,cumreturndow,cumreturnsp500
0,2019-09-03,-1.0,-0.723623,1.007236,1.010842,1.009091
1,2019-09-04,-1.0,10.982052,0.896621,1.023993,1.023361
2,2019-09-05,-1.0,13.833619,0.772586,1.024925,1.026014
3,2019-09-06,-1.0,13.597485,0.667534,1.024829,1.027471
4,2019-09-09,-1.0,32.908813,0.447856,1.025159,1.030301
5,2019-09-10,-1.0,61.744503,0.171330,1.032571,1.039016
6,2019-09-11,-1.0,-3.842127,0.177912,1.035544,1.040755
7,2019-09-12,-1.0,33.907968,0.117586,1.034794,1.042174
8,2019-09-13,-1.0,-9.062491,0.128242,1.031549,1.036710
9,2019-09-16,-1.0,-15.393375,0.147983,1.034212,1.038011


In [164]:
#hide_input
import altair as alt
a1 = alt.Chart(df_a).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1b:Q',
    size='cumreturn1b',
    
).properties(
    width=1000,
    height=1000,
    title='10QPortfolio (blue) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_a).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_a).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

#a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
#    x='Date:T',
#    y='cumreturn1a:Q',

#).properties(
#    width=1000,
#    height=1000
#)

#a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
#    x='Date:T',
#    y='cumreturn1b:Q',

#).properties(
#    width=1000,
#    height=1000
#)

a1 + a2 + a3# + a4 + a5

alt.LayerChart(...)